Рубашевская Анастасия 3.9
Тема: Предсказание возраста краба на основе метрических данных методами регрессии

In [2]:
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.model_selection import train_test_split # импортируем из библиотеки  sklearn функцию train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TheilSenRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import HuberRegressor
# импортируем mean_squared_error из библиотеки Scikit-learn, которая используется для вычисления среднеквадратичной ошибки
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
import pandas # библиотека для анализа данных
import numpy # библиотека, которая поддерживает многомерные массивы и математические функции
from collections import Counter

In [3]:
numpy.seterr(invalid='ignore') # игнорирует numpy-евские предупреждения о преобразовании данных
data = pandas.read_csv("CrabAgePrediction.csv") # читаем csv-файл через библиотеку pandas CrabAgePrediction.csv
replace_dict = {'I': 1, 'F': 2, 'M': 3} # преобразовываем строчные данные (пол краба) в цифру
data = data.replace(replace_dict)
# приводим данные csv файла к структуре датасета как в sklearn
dataset = {'data': data[[data.columns[i] for i in range(8)]].to_numpy(), # данные для обучения
             'target': data[data.columns[8]].to_numpy(dtype=numpy.int64), # столбец возраста краба (счет от 0 до 8)
             'DESCR': '\n\nCrabAgePrediction'} # название датасета

print("-----------------------------------------------------------------")
print("Датасет: ", dataset['DESCR'].splitlines()[2]) #вывод названия датасета
print(Counter(data.iloc[:, 8]))
print("-----------------------------------------------------------------")
# деление на тестовую и тренировочные части, где 70% на тренировку
X_train, X_test, Y_train, Y_test = train_test_split(dataset['data'], dataset['target'], train_size = 0.7)
# пропускаем данные через полиномиальную регрессию. degree - степень полинома
Poly_train, Poly_test = PolynomialFeatures(degree = 3).fit_transform(X_train), PolynomialFeatures(degree = 3).fit_transform(X_test)
# массив всех регрессионных моделей
AllModels = ['KNeighborsRegressor', 'RadiusNeighborsRegressor', 'LinearSVR', 'SVR', 'LinearRegression', 'Ridge', 'Lasso', 'ElasticNet',
             'LinearRegression + Полиномиальная регрессия', 'Ridge + Полиномиальная регрессия', 'Lasso + Полиномиальная регрессия', 'ElasticNet + Полиномиальная регрессия',
             'Полиномиальная регрессия + TheilSenRegressor', 'Полиномиальная регрессия + HuberRegressor', 'RANSACRegressor + LinearRegression', 'RANSACRegressor + Ridge',
             'RANSACRegressor + Lasso', 'RANSACRegressor + ElasticNet', 'RANSACRegressor + LinearRegression + Полиномиальная регрессия',
             'RANSACRegressor + Ridge + Полиномиальная регрессия', 'RANSACRegressor + Lasso + Полиномиальная регрессия', 'RANSACRegressor + ElasticNet + Полиномиальная регрессия']
MSEOfModels = numpy.array([]) # пустой массив лучших

-----------------------------------------------------------------
Датасет:  CrabAgePrediction
Counter({9: 640, 10: 598, 8: 520, 11: 459, 7: 357, 12: 248, 6: 241, 13: 195, 14: 122, 5: 107, 15: 96, 16: 62, 17: 54, 4: 53, 18: 38, 19: 31, 20: 23, 21: 14, 3: 13, 23: 7, 22: 6, 24: 2, 27: 2, 26: 1, 2: 1, 1: 1, 29: 1, 25: 1})
-----------------------------------------------------------------


In [4]:
# KNeighborsRegressor
print("KNeighborsRegressor")
BestKNeighbor = numpy.array([]) # создаю пустой массив для среднеквадратичных ошибок
# Проход по числу соседей от 1 до 19 включительно
for n in range(1, 20):
   # Для каждого значения n создается модель KNeighborsRegressor с соответствующим числом соседей
   model = KNeighborsRegressor(n_neighbors = n)
   # обучение модели по тренировочным данным
   model.fit(X_train, Y_train)
   """Вычисляется среднеквадратичная ошибка на тестовых данных (Y_test)
   для текущей модели с числом соседей n и добавляется в массив BestKNeighbor."""
   BestKNeighbor = numpy.append(BestKNeighbor, mean_squared_error(Y_test, model.predict(X_test)))

print(BestKNeighbor) #вывод массива среднеквадратичных ошибок
# Вывод лучшего числа соседей, которое соответствует минимальной среднеквадратической ошибке
print("Лучшее число соседей", numpy.argmin(BestKNeighbor) + 1, "с среднеквадратичной ошибкой", BestKNeighbor[numpy.argmin(BestKNeighbor)])
MSEOfModels = numpy.append(MSEOfModels, BestKNeighbor[numpy.argmin(BestKNeighbor)])
print("-----------------------------------------------------------------")


KNeighborsRegressor
[8.04537671 6.33154966 5.68807078 5.37307363 5.16044521 5.12611777
 5.14477914 5.04525631 4.97878615 4.9322089  4.93841277 4.89484042
 4.85482188 4.8542118  4.85248478 4.78339376 4.79614696 4.76175112
 4.75335351]
Лучшее число соседей 19 с среднеквадратичной ошибкой 4.753353508139491
-----------------------------------------------------------------


In [5]:
# RadiusNeighborsRegressor
print("RadiusNeighborsRegressor")
BestRadiusNeighbor = numpy.array([])
# Цикл прохода по радиусам от 1 до 19  включительно
for r in range(1, 20):
   model = RadiusNeighborsRegressor(radius = r) #создание модели с указанным радиусом r
   # Радиус здесь служит в качестве параметра, определяющего радиус окрестности, в пределах которой находятся соседи
   model.fit(X_train, Y_train) # обучение модели по тренировочным данным
   """ Вычисляется среднеквадратичная ошибка на тестовых данных (Y_test)
   для текущей модели с радиусом r и добавляется в массив BestRadiusNeighbor.
   Здесь используется функция mean_squared_error для расчета среднеквадратичной ошибки."""
   BestRadiusNeighbor = numpy.append(BestRadiusNeighbor, mean_squared_error(Y_test, model.predict(X_test)))

print(BestRadiusNeighbor) # Выводим массив среднеквадратичных ошибок
# Вывой информации о лучшем радиусе, который соответствует наименьшей среднеквадратической ошибке
print("Лучший радиус", numpy.argmin(BestRadiusNeighbor) + 1, "с среднеквадратичной ошибкой", BestRadiusNeighbor[numpy.argmin(BestRadiusNeighbor)])
MSEOfModels = numpy.append(MSEOfModels, BestRadiusNeighbor[numpy.argmin(BestRadiusNeighbor)])
print("-----------------------------------------------------------------")

RadiusNeighborsRegressor
[1.48582198e+37 2.76770761e+36 1.16535057e+36 4.37006464e+35
 7.28344107e+34 7.28344107e+34 7.28344107e+34 7.28344107e+34
 7.28344107e+34 6.57177385e+00 6.66816310e+00 6.76066285e+00
 6.83218501e+00 6.93489538e+00 7.01643947e+00 7.09995194e+00
 7.18575510e+00 7.27047148e+00 7.36569422e+00]
Лучший радиус 10 с среднеквадратичной ошибкой 6.571773848351284
-----------------------------------------------------------------


In [6]:
# LinearSVR
print("LinearSVR")
model = LinearSVR(dual = True, max_iter = 100000)
"""dual=True: Это логический параметр, который указывает, следует ли использовать двойственную или прямую форму оптимизации.
Параметр dual влияет на способ оптимизации алгоритма. Если dual=True, алгоритм оптимизирует двойственную форму задачи оптимизации.
max_iter = 100000 - максимальное число итераций определяется для того, чтобы модель точно доучилась до конца"""

model.fit(X_train, Y_train) # обучение модели на тренировочных данных

"""Расчитываем среднеквадратическую ошибку между фактическими целевыми значениями тестовых данных Y_test
и предсказанными значениями, полученными с помощью метода predict(), модели на тестовых данных X_test.
Она оценивает, насколько хорошо модель предсказывает целевую переменную."""
mse = mean_squared_error(Y_test, model.predict(X_test))
print("Cреднеквадратичная ошибка linearSVR", mse)
MSEOfModels = numpy.append(MSEOfModels, mse)
print("-----------------------------------------------------------------")

LinearSVR
Cреднеквадратичная ошибка linearSVR 4.978322485558872
-----------------------------------------------------------------


In [7]:
# SVR
print("SVR")
kernels = ['linear', 'poly', 'rbf', 'sigmoid'] # массив ядер
BestSVR = numpy.array([]) # пустой массив среднеквадратических ошибок
# Цикл прохода по ядрам
for kernel in kernels:
   model = SVR(kernel = kernel) # создание модели с указанным ядром
   model.fit(X_train, Y_train) # обучение модели на тренировочных данных
   # заполняем массив среднеквадратических ошибок нашими значениями
   BestSVR = numpy.append(BestSVR, mean_squared_error(Y_test, model.predict(X_test)))
# вывод сперва массива ядер, затем массива ошибок, чтобы понимать какому ядру соответствует какая ошибка
print(kernels)
print(BestSVR)
# Вывод лучшего ядра, которое соответствует минимальной квадратической ошибке
print("Лучшее ядро", kernels[numpy.argmin(BestSVR)], "с среднеквадратичной ошибкой", BestSVR[numpy.argmin(BestSVR)])
MSEOfModels = numpy.append(MSEOfModels, BestSVR[numpy.argmin(BestSVR)])
print("-----------------------------------------------------------------")

SVR
['linear', 'poly', 'rbf', 'sigmoid']
[4.98950221e+00 6.97195792e+00 4.94312951e+00 9.47956355e+03]
Лучшее ядро rbf с среднеквадратичной ошибкой 4.943129509581358
-----------------------------------------------------------------


In [8]:
# LinearRegression
print("LinearRegression")
model = LinearRegression() # создается модель линейной регрессии
model.fit(X_train, Y_train) # обучение модели на тренировочных данных
# Расчет среднеквадратической ошибки и вывод
mse = mean_squared_error(Y_test, model.predict(X_test))
print("Cреднеквадратичная ошибка LinearRegression", mse)
MSEOfModels = numpy.append(MSEOfModels, mse)
print("-----------------------------------------------------------------")


LinearRegression
Cреднеквадратичная ошибка LinearRegression 4.872571838711208
-----------------------------------------------------------------


In [9]:
# Ridge
print("Ridge")
BestRidge = numpy.array([]) # создаем пустой массив среднеквадратических ошибок
# Цикл прохода по значениям а от 0 до 20 включительно
for a in range(21):
   # параметр регуляризации alpha (позволяет предотвратить переобучение модели) вычисляется как а/2 - таким образом в цикле делаю шаг 0.5
   alpha = a / 2
   model = Ridge(alpha = alpha) # создаем модель с указанным значением параметра alpha
   model.fit(X_train, Y_train) # обучение модели по тренировочным данным
   # Заполняем пустой массив ошибок расчетными значениями
   BestRidge = numpy.append(BestRidge, mean_squared_error(Y_test, model.predict(X_test)))

print(BestRidge) # Вывод массива ошибок
# Вывод информации о лучшем значении alpha, который соответствует наименьшей среднеквадратической ошибке
print("Лучший alpha в Ridge", numpy.argmin(BestRidge) / 2, "с среднеквадратичной ошибкой", BestRidge[numpy.argmin(BestRidge)])
MSEOfModels = numpy.append(MSEOfModels, BestRidge[numpy.argmin(BestRidge)])
print("-----------------------------------------------------------------")

Ridge
[4.87257184 4.87421111 4.876605   4.87923909 4.88190456 4.88451388
 4.88703175 4.88944655 4.89175763 4.89396931 4.89608807 4.89812103
 4.9000753  4.90195762 4.90377424 4.90553083 4.90723254 4.908884
 4.91048935 4.91205232 4.91357624]
Лучший alpha в Ridge 0.0 с среднеквадратичной ошибкой 4.87257183871121
-----------------------------------------------------------------


In [10]:
# Lasso
print("Lasso")
BestLasso = numpy.array([])
# Цикл прохода по значениям а от 1 до 20 включительно
for a in range(1, 21):
   # параметр регуляризации alpha (позволяет предотвратить переобучение модели) вычисляется как а/2 - таким образом в цикле делаю шаг 0.5
   alpha = a / 2
   model = Lasso(alpha = alpha) # создание модели с указанным параметром alpha
   model.fit(X_train, Y_train) # обучение модели по тренировочным данным
   # Заполняем массив ошибок расчетными значениями
   BestLasso = numpy.append(BestLasso, mean_squared_error(Y_test, model.predict(X_test)))
# вывод массива среднеквадратических ошибок
print(BestLasso)
# Вывод информации о лучшем параметре alpha, который соответствует минимальной среднеквадратической ошибки
print("Лучший alpha в Lasso", (numpy.argmin(BestLasso) + 1) / 2, "с среднеквадратичной ошибкой", BestLasso[numpy.argmin(BestLasso)])
MSEOfModels = numpy.append(MSEOfModels, BestLasso[numpy.argmin(BestLasso)])
print("-----------------------------------------------------------------")


Lasso
[5.46520725 5.94326123 6.51448886 7.09674119 7.11245955 7.1255724
 7.14141201 7.15997837 7.1812715  7.20529138 7.23203803 7.26151143
 7.29371159 7.32863851 7.36629219 7.40667263 7.44977983 7.49561378
 7.5441745  7.59546197]
Лучший alpha в Lasso 0.5 с среднеквадратичной ошибкой 5.465207251515108
-----------------------------------------------------------------


In [11]:
# ElasticNet
print("ElasticNet")
BestElastic = numpy.array([])

for ratio in range(10):
   alpha = ratio * 2 + 1
   l1 = (10 - ratio) / 10

   model = ElasticNet(alpha = alpha, l1_ratio = l1, max_iter = 100000)
   model.fit(X_train, Y_train)
   BestElastic = numpy.append(BestElastic, mean_squared_error(Y_test, model.predict(X_test)))

print(BestElastic)
print("Лучшие alpha и l1_ratio в ElasticNet", numpy.argmin(BestElastic) * 2 + 1, (10 - numpy.argmin(BestElastic)) / 10, "с среднеквадратичной ошибкой", BestElastic[numpy.argmin(BestElastic)])
MSEOfModels = numpy.append(MSEOfModels, BestElastic[numpy.argmin(BestElastic)])
print("-----------------------------------------------------------------")

ElasticNet
[5.94326123 7.1182397  7.16418038 7.21079771 7.24603565 7.26253123
 7.25757499 7.23281234 7.19372824 7.07196159]
Лучшие alpha и l1_ratio в ElasticNet 1 1.0 с среднеквадратичной ошибкой 5.943261227682373
-----------------------------------------------------------------


In [12]:
# Линейная регрессия + Полиномиальная регрессия
print("Линейная регрессия + Полиномиальная регрессия")
model = LinearRegression() # создание модели линейной регрессии
model.fit(Poly_train, Y_train) # обучение модели по тренировочным полимеальным данным и тренировочным Y
# расчет среднеквадратической ошибки и ее вывод
mse = mean_squared_error(Y_test, model.predict(Poly_test))
print("Cреднеквадратичная ошибка Полиномиальная + LinearRegression", mse)
MSEOfModels = numpy.append(MSEOfModels, mse)
print("-----------------------------------------------------------------")

Линейная регрессия + Полиномиальная регрессия
Cреднеквадратичная ошибка Полиномиальная + LinearRegression 5.108614410448293
-----------------------------------------------------------------


In [13]:
# Ridge + Полиномиальная регрессия
print("Ridge + Полиномиальная регрессия")
BestRidge = numpy.array([]) # создание пустого массива среднеквадратических ошибок
# Цикл прохода по значениям а от 0 до 20 включительно
for a in range(21):
   # параметр регуляризации alpha (позволяет избежать переобучение модели) вычисляется как а/2 - таким образом в цикле делаю шаг 0.5
   alpha = a / 2
   model = Ridge(alpha = alpha) # создание модели с указанным alpha
   model.fit(Poly_train, Y_train) # обучение модели по тренировочным полиномеальным данным и по тренировочным Y
   # заполняем массив ошибок расчетными значениями
   BestRidge = numpy.append(BestRidge, mean_squared_error(Y_test, model.predict(Poly_test)))
# вывод массива ошибок
print(BestRidge)
# вывод информации о лучшем alpha
print("Лучший alpha в Полиномиальной + Ridge", numpy.argmin(BestRidge) / 2, "с среднеквадратичной ошибкой", BestRidge[numpy.argmin(BestRidge)])
MSEOfModels = numpy.append(MSEOfModels, BestRidge[numpy.argmin(BestRidge)])
print("-----------------------------------------------------------------")

Ridge + Полиномиальная регрессия
[5.10922925 5.13751398 5.12688505 5.11502299 5.10374211 5.09354079
 5.0844778  5.07646894 5.0693915  5.06312171 5.05754736 5.05257112
 5.04811007 5.04409418 5.04046438 5.0371709  5.0341716  5.03143078
 5.02891802 5.0266073  5.0244763 ]
Лучший alpha в Полиномиальной + Ridge 10.0 с среднеквадратичной ошибкой 5.024476302293616
-----------------------------------------------------------------


In [14]:
# Lasso + Полиномиальная регрессия
print("Lasso + Полиномиальная регрессия")
BestLasso = numpy.array([]) # создаем пустой массив
# Параметр регуляризации берем побольше, чтобы избежать предупреждений о малой регуляризации
# Цикл прохода по значениям а от 3000 до 3019 включительно
for a in range(3000, 3020):
   model = Lasso(alpha = a) # создание модели с указанным параметром alpha
   model.fit(Poly_train, Y_train) # обучение модели по тренировочным полиномеаьным данным и тренировочным Y
   # заполняем пустой массив ошибок расчетными значениями среднеквадратических ошибок
   BestLasso = numpy.append(BestLasso, mean_squared_error(Y_test, model.predict(Poly_test)))
# Вывод массива ошибок и информации о лучшем alpha, который соответствует минимальной среднеквадратической ошибке
print(BestLasso)
print("Лучший alpha в Полиномиальной + Lasso", numpy.argmin(BestLasso) + 3000, "с среднеквадратичной ошибкой", BestLasso[numpy.argmin(BestLasso)])
MSEOfModels = numpy.append(MSEOfModels, BestLasso[numpy.argmin(BestLasso)])
print("-----------------------------------------------------------------")

Lasso + Полиномиальная регрессия
[8.52172307 8.52212165 8.5225205  8.52291962 8.52331901 8.52371868
 8.52411862 8.52451882 8.5249193  8.52532006 8.52572108 8.52612237
 8.52652394 8.52692578 8.52732789 8.52773027 8.52813292 8.52853585
 8.52893905 8.52934251]
Лучший alpha в Полиномиальной + Lasso 3000 с среднеквадратичной ошибкой 8.521723070568992
-----------------------------------------------------------------


In [15]:
#  ElasticNet + Полиномиальная регрессия
print("ElasticNet + Полиномиальная регрессия")
BestElastic = numpy.array([]) # создаем пустой массив среднеквадратических ошибок
"""Коэффициент ratio определяет степень принадлежности метода ElasticNet либо к Lasso, либо к Ridge.
Lasso - если aplha большой, Ridge - если alpha маленький
Lasso - l1_ratio < 0.5, Ridge - l1_ratio > 0.5"""
# Цикл прохода по параметру ratio от 0 от 9 включительно
for ratio in range(10):
   """ значение цикла (ratio) умножается на 333 и к результату прибавляем 10 из-за того что параметр alpha может меняться до примерно 3000
   и нужно поделить этот промежуток на 10 частей. А параметр альфа чем больше, тем ближе нам метод к lasso.
   Нам нужно знать к чему стремится метод у конкретной модели"""
   alpha = ratio * 333 + 10
   """параметр ratio может изменяться от 0 до 1 при этом границей между lasso и ridge является 0.5. Промежуток делится на 10 частей.
   Вычитаем значение нашего цикла от максимального значения цикла, затем делим на 10 чтобы понять к чему стремится параеметр ratio
   (в нашей ситуации при шаге 0.1)"""
   l1 = (10 - ratio) / 10

   model = ElasticNet(alpha = alpha, l1_ratio = l1, max_iter = 100000) # создание модели
   model.fit(Poly_train, Y_train) # обучение модели по тренировочным полиномеальным данным и по тренировочным Y
   # Расчитываются значения среднеквадратической ошибки и заносятся в массив ошибок
   BestElastic = numpy.append(BestElastic, mean_squared_error(Y_test, model.predict(Poly_test)))
# вывод массива ошибок и информации о лучших alpha и ratio, которые соответствуют минимальной среднеквадратической ошибке
print(BestElastic)
print("Лучшие alpha и l1_ratio в Полиномиальной + ElasticNet", numpy.argmin(BestElastic) * 333 + 10, (10 - numpy.argmin(BestElastic)) / 10, "с среднеквадратичной ошибкой", BestElastic[numpy.argmin(BestElastic)])
MSEOfModels = numpy.append(MSEOfModels, BestElastic[numpy.argmin(BestElastic)])
print("-----------------------------------------------------------------")

ElasticNet + Полиномиальная регрессия
[5.60404868 7.73440554 8.13252055 8.12485869 8.11986623 8.1169497
 8.12004665 8.12480369 8.13280495 7.72186724]
Лучшие alpha и l1_ratio в Полиномиальной + ElasticNet 10 1.0 с среднеквадратичной ошибкой 5.604048683979123
-----------------------------------------------------------------


In [16]:
# Полиномиальная + TheilSenRegressor
print("Полиномиальная + TheilSenRegressor")
model = TheilSenRegressor(random_state = 100000, max_iter = 1000000, n_subsamples = 500, max_subpopulation = 150)
""" n_subsamples = 500 определяет количество подмножеств со с рандомными значениями
(которые генерятся при помощи random_state) для обучения.
max_subpopulation = 150 устанавливает максимальный размер подмножеств в выборке."""
model.fit(Poly_train, Y_train.ravel()) # обучение модели на тренировочных полиномеальных данных и тренировочных Y
# ravel() функция которая превращает многомерный массив в одномерный, тк того требует TheilSenRegressor
# подсчет и вывод среднеквадратичной ошибки
mse = mean_squared_error(Y_test, model.predict(Poly_test))
print("Cреднеквадратичная ошибка Полиномиальная + TheilSenRegressor", mse)
MSEOfModels = numpy.append(MSEOfModels, mse)
print("-----------------------------------------------------------------")

Полиномиальная + TheilSenRegressor
Cреднеквадратичная ошибка Полиномиальная + TheilSenRegressor 9.076665066668165
-----------------------------------------------------------------


In [17]:
# Полиномиальная + HuberRegressor
print("Полиномиальная + HuberRegressor")
# Создание и настройка модели HuberRegressor.
""" epsilon - параметр, который определяет границу между использованием квадратичной функции потерь и абсолютной функции потерь.
Если абсолютное значение отклонения меньше epsilon, то используется квадратичная функция потерь, в противном случае - абсолютная.
Значение по умолчанию обычно составляет 1.35. Параметр epsilon влияет на то, на сколько модель устойчива к выбросам (шумам).
Чем больше значение epsilon, тем лучше."""
model = HuberRegressor(epsilon=100, max_iter=10000, alpha=0.0001)
model.fit(Poly_train, Y_train) # обучение модели
#  оценка среднеквадратичной ошибки модели на тестовых данных Poly_test.
#Эта ошибка измеряет, насколько сильно предсказания модели отклоняются от фактических значений Y_test.
mse = mean_squared_error(Y_test, model.predict(Poly_test))
print("Cреднеквадратичная ошибка Полиномиальная + HuberRegressor", mse)
MSEOfModels = numpy.append(MSEOfModels, mse)
print("-----------------------------------------------------------------")

Полиномиальная + HuberRegressor
Cреднеквадратичная ошибка Полиномиальная + HuberRegressor 6.171294280275931
-----------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [18]:
# RANSACRegressor + LinearRegression
print("RANSACRegressor + LinearRegression")
""" min_samples - Определяет минимальное количество точек, которые будут использоваться для подгонки модели в каждой итерации RANSAC.
В данном случае, примерно 70% тестовой части данных (800 точек).
residual_threshold: Это максимальное отклонение точек от модели, которое считается приемлемым.
Точки, отклонение которых превышает это значение, не учитываются при подгонке модели."""

model = RANSACRegressor(LinearRegression(), min_samples = 800, residual_threshold = 1.0) # создание модели базе линейной регрессии
model.fit(X_train, Y_train) # обучение модели на тренировочных значениях
# Расчет ошибки и вывод
mse = mean_squared_error(Y_test, model.predict(X_test))
print("Cреднеквадратичная ошибка RANSACRegressor + LinearRegression", mse)
MSEOfModels = numpy.append(MSEOfModels, mse)
print("-----------------------------------------------------------------")

RANSACRegressor + LinearRegression
Cреднеквадратичная ошибка RANSACRegressor + LinearRegression 4.999895347473191
-----------------------------------------------------------------


In [19]:
# RANSACRegressor + Ridge
print("RANSACRegressor + Ridge")
BestRidge = numpy.array([]) # создание пустого массива ошибок
""" min_samples - Определяет минимальное количество точек, которые будут использоваться для подгонки модели в каждой итерации RANSAC.
В данном случае, примерно 70% тестовой части данных (800 точек).
residual_threshold: Это максимальное отклонение точек от модели, которое считается приемлемым.
Точки, отклонение которых превышает это значение, не учитываются при подгонке модели."""
# Цикл прохода по а от 0 до 20 включительно
for a in range(21):
   alpha = a / 2
   model = RANSACRegressor(Ridge(alpha = alpha) , min_samples = 800, residual_threshold = 1.0) # создание модели на базе Ridge с указанием конкретного alpha
   model.fit(X_train, Y_train) # обучение модели
   BestRidge = numpy.append(BestRidge, mean_squared_error(Y_test, model.predict(X_test))) # расчет ошибки и заполнение массива ошибок нашими значениями

print(BestRidge) # вывод массива
# Вывод информации о лучшем параметре alpha, который соответствует минимальной среднеквадратической ошибке
print("Лучший alpha в RANSACRegressor + Ridge", numpy.argmin(BestRidge) / 2, "с среднеквадратичной ошибкой", BestRidge[numpy.argmin(BestRidge)])
MSEOfModels = numpy.append(MSEOfModels, BestRidge[numpy.argmin(BestRidge)])
print("-----------------------------------------------------------------")

RANSACRegressor + Ridge
[4.8260326  4.85636819 4.94765812 5.02059067 4.95274126 4.98788135
 5.00078232 4.97386389 4.98929203 4.9969265  5.03528743 5.0073467
 5.03858318 5.02754497 5.06723254 5.04000104 5.0441169  5.08596761
 5.08576992 5.09368427 5.05254034]
Лучший alpha в RANSACRegressor + Ridge 0.0 с среднеквадратичной ошибкой 4.8260325982604275
-----------------------------------------------------------------


In [20]:
# RANSACRegressor + Lasso
print("RANSACRegressor + Lasso")
BestLasso = numpy.array([]) # создание пустого массива ошибок
""" min_samples - Определяет минимальное количество точек, которые будут использоваться для подгонки модели в каждой итерации RANSAC.
В данном случае, примерно 70% тестовой части данных (800 точек).
residual_threshold: Это максимальное отклонение точек от модели, которое считается приемлемым.
Точки, отклонение которых превышает это значение, не учитываются при подгонке модели."""
# Цикл прохода по а от 1 до 20 включительно
for a in range(1, 21):
   alpha = a / 2
   model = RANSACRegressor(Lasso(alpha = alpha), min_samples = 800, residual_threshold = 1.0) # создание модели на базе lasso с указанием парметра alpha
   model.fit(X_train, Y_train) # обучение модели
   BestLasso = numpy.append(BestLasso, mean_squared_error(Y_test, model.predict(X_test))) # расчет ошибки и заполнение массива ошибок

print(BestLasso) # вывод массива на экран и вывод информации о лучшем alpha, которым соответствует минимальной среднеквадратической ошибки
print("Лучший alpha в RANSACRegressor + Lasso", (numpy.argmin(BestLasso) + 1) / 2, "с среднеквадратичной ошибкой", BestLasso[numpy.argmin(BestLasso)])
MSEOfModels = numpy.append(MSEOfModels, BestLasso[numpy.argmin(BestLasso)])
print("-----------------------------------------------------------------")

RANSACRegressor + Lasso
[6.06364831 7.18797641 7.23862408 7.30303294 7.36537453 7.4712457
 7.63366764 7.57091037 7.59189223 7.84502678 8.10745428 7.96580854
 8.65383973 8.78442399 8.91842219 8.99260233 9.07819954 9.77453865
 9.38659034 9.79847301]
Лучший alpha в RANSACRegressor + Lasso 0.5 с среднеквадратичной ошибкой 6.063648310699348
-----------------------------------------------------------------


In [21]:
# RANSACRegressor + ElasticNet
print("RANSACRegressor + ElasticNet")
#Lasso - aplha is big, Ridge - alpha is small
#Lasso - li_ratio < 0.5, Ridge - l1_ratio > 0.5
""" min_samples - Определяет минимальное количество точек, которые будут использоваться для подгонки модели в каждой итерации RANSAC.
В данном случае 1000 точек.
residual_threshold: Это максимальное отклонение точек от модели, которое считается приемлемым.
Точки, отклонение которых превышает это значение, не учитываются при подгонке модели."""
BestElastic = numpy.array([]) # создание пустого массива ошибок
#  Цикл прохода по параметру ratio от 0 до 9 включительно
for ratio in range(10):
   alpha = ratio * 2 + 1 # Вычисляется значение alpha на основе текущего значения ratio
   l1 = (10 - ratio) / 10 # Вычисляется значение l1_ratio на основе текущего значения ratio

   model = RANSACRegressor(ElasticNet(alpha = alpha, l1_ratio = l1), min_samples = 1000, residual_threshold = 0.5) # создание модели на базе ElasticNet
   model.fit(X_train, Y_train) # обучение модели
   BestElastic = numpy.append(BestElastic, mean_squared_error(Y_test, model.predict(X_test))) # расчет среднеквадратической ошибки и заполнение массива ошибок
# вывод массива ошибок и информации о лучшем alpha и ratio, которые соответствуют минимальной среднеквадратической ошибки
print(BestElastic)
print("Лучшие alpha и l1_ratio в RANSACRegressor + ElasticNet", numpy.argmin(BestElastic) * 2 + 1, (10 - numpy.argmin(BestElastic)) / 10, "с среднеквадратичной ошибкой", BestElastic[numpy.argmin(BestElastic)])
MSEOfModels = numpy.append(MSEOfModels, BestElastic[numpy.argmin(BestElastic)])
print("-----------------------------------------------------------------")


RANSACRegressor + ElasticNet
[7.12122579 7.4395226  7.76097723 7.98475921 8.07094466 8.18001509
 8.06671598 7.98484729 7.80445398 7.60546984]
Лучшие alpha и l1_ratio в RANSACRegressor + ElasticNet 1 1.0 с среднеквадратичной ошибкой 7.121225786657732
-----------------------------------------------------------------


In [22]:
# RANSACRegressor + LinearRegression + Полиномиальная регрессия
print("RANSACRegressor + LinearRegression + Полиномиальная регрессия ")
""" min_samples - Определяет минимальное количество точек, которые будут использоваться для подгонки модели в каждой итерации RANSAC.
В данном случае 1000 точек.
residual_threshold: Это максимальное отклонение точек от модели, которое считается приемлемым.
Точки, отклонение которых превышает это значение, не учитываются при подгонке модели."""
model = RANSACRegressor(LinearRegression(), min_samples = 1000, residual_threshold = 0.5) # создание модели на базе линейной регрессии
model.fit(Poly_train, Y_train) # обучение модели по тренировочным полиномеальным данным и по тренировочным Y
# расчет среднеквадратической ошибки и вывод ее на экран, а также вывод на экран информации о лушем значении
mse = mean_squared_error(Y_test, model.predict(Poly_test))
print("Cреднеквадратичная ошибка Полиномиальная + RANSACRegressor + LinearRegression", mse)
MSEOfModels = numpy.append(MSEOfModels, mse)
print("-----------------------------------------------------------------")

RANSACRegressor + LinearRegression + Полиномиальная регрессия 
Cреднеквадратичная ошибка Полиномиальная + RANSACRegressor + LinearRegression 10.138533048124463
-----------------------------------------------------------------


In [23]:
# RANSACRegressor + Ridge + Полиномиальная регрессия
print("RANSACRegressor + Ridge + Полиномиальная регрессия ")
BestRidge = numpy.array([]) # создаем пустой массив ошибок
# Цикл прохода по а от 0 до 20 включительно
for a in range(21):
   alpha = a / 2 # вычисление значения аlpha на основе значения нашего цикла
   model = RANSACRegressor(Ridge(alpha = alpha) , min_samples = 800, residual_threshold = 1.0) # создание модели на основе Ridge с указанием конкретного alpha
   model.fit(Poly_train, Y_train) # обучение модели на основе тренировочных полиномеальных данных и тренировочных Y
   BestRidge = numpy.append(BestRidge, mean_squared_error(Y_test, model.predict(Poly_test))) # расчет ошибки и заполнение массива ошибок

print(BestRidge) # вывод массива на экран
# Вывод информации о лучшем alpha, который соответствует наименьшей среднеквадратической ошибке
print("Лучший alpha в Полиномиальная + RANSACRegressor + Ridge", numpy.argmin(BestRidge) / 2, "с среднеквадратичной ошибкой", BestRidge[numpy.argmin(BestRidge)])
MSEOfModels = numpy.append(MSEOfModels, BestRidge[numpy.argmin(BestRidge)])
print("-----------------------------------------------------------------")

RANSACRegressor + Ridge + Полиномиальная регрессия 
[48.71475873  7.46234997  6.19094859  8.21186181  8.73802387  7.54786697
  8.42487083  6.46380028  5.77031247  6.01982516  6.34544622  5.48361831
  7.01488433  6.74106465  8.37121957  9.72923874  8.43663142  6.0484898
  6.14543823 13.24330609  5.79903911]
Лучший alpha в Полиномиальная + RANSACRegressor + Ridge 5.5 с среднеквадратичной ошибкой 5.483618314465247
-----------------------------------------------------------------


In [24]:
# RANSACRegressor + Lasso + Полиномиальная регрессия
print("RANSACRegressor + Lasso + Полиномиальная регрессия")
BestLasso = numpy.array([])# создаем пустой массив ошибок
# Цикл прохода по а от 5000 до 5019 включительно
for a in range(5000, 5020):
   alpha = a / 2 # вычисление значения аlpha на основе значения нашего цикла
   model = RANSACRegressor(Lasso(alpha = alpha), min_samples = 800, residual_threshold = 1.0) # создание модели на основе lasso с указанием конкретного alpha
   model.fit(Poly_train, Y_train) # обучение модели на основе тренировочных полиномеальных данных и тренировочных Y
   BestLasso = numpy.append(BestLasso, mean_squared_error(Y_test, model.predict(Poly_test))) # расчет ошибки и заполнение массива ошибок

print(BestLasso) # вывод массива на экран
# Вывод информации о лучшем alpha, который соответствует наименьшей среднеквадратической ошибке
print("Лучший alpha в Полиномиальная + RANSACRegressor + Lasso", (numpy.argmin(BestLasso) + 5000) / 2, "с среднеквадратичной ошибкой", BestLasso[numpy.argmin(BestLasso)])
MSEOfModels = numpy.append(MSEOfModels, BestLasso[numpy.argmin(BestLasso)])
print("-----------------------------------------------------------------")

RANSACRegressor + Lasso + Полиномиальная регрессия
[8.71327603 8.72991666 8.7166779  9.09512097 8.82777661 8.67112396
 8.71176203 8.64620413 8.73434019 8.6599164  8.73181421 8.75750433
 8.70334459 8.82381281 8.65371693 8.71977727 8.73907934 8.76109519
 8.74178087 8.72368302]
Лучший alpha в Полиномиальная + RANSACRegressor + Lasso 2503.5 с среднеквадратичной ошибкой 8.64620412625933
-----------------------------------------------------------------


In [25]:
# RANSACRegressor + ElasticNet + Полиномиальная регрессия
print("RANSACRegressor + ElasticNet + Полиномиальная регрессия")
#Lasso - aplha is big, Ridge - alpha is small
#Lasso - li_ratio < 0.5, Ridge - l1_ratio > 0.5

BestElastic = numpy.array([]) # создаем пустой массив ошибок
# Цикл прохода по ratio от 0 до 9 включительно
for ratio in range(10):
   alpha = ratio * 1000 + 2000
   l1 = (10 - ratio) / 10

   model = RANSACRegressor(ElasticNet(alpha = alpha, l1_ratio = l1), min_samples = 1000, residual_threshold = 0.5) # создание модели ElasticNet с указанием конкретных параметров
   model.fit(Poly_train, Y_train) # обучение модели
   BestElastic = numpy.append(BestElastic, mean_squared_error(Y_test, model.predict(Poly_test))) # заполняем массив значениями ошибок.
   #Далее выводим его и лучший alpha и ratio при минимальной ошибки

print(BestElastic)
print("Лучшие alpha и l1_ratio в Полиномиальная + RANSACRegressor + ElasticNet", numpy.argmin(BestElastic) * 1000 + 2000, (10 - numpy.argmin(BestElastic)) / 10, "с среднеквадратичной ошибкой", BestElastic[numpy.argmin(BestElastic)])
MSEOfModels = numpy.append(MSEOfModels, BestElastic[numpy.argmin(BestElastic)])
print("-----------------------------------------------------------------")


RANSACRegressor + ElasticNet + Полиномиальная регрессия


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.006e+01, tolerance: 1.020e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.089e+00, tolerance: 9.886e-01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.022e+00, tolerance: 1.124e

[8.67595642 8.67414858 8.75053092 8.73635735 8.68669442 8.76280624
 8.73837037 8.61749138 8.64000306 8.38595006]
Лучшие alpha и l1_ratio в Полиномиальная + RANSACRegressor + ElasticNet 11000 0.1 с среднеквадратичной ошибкой 8.385950057246832
-----------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.107e+00, tolerance: 9.648e-01
  model = cd_fast.enet_coordinate_descent(


In [26]:
# Итоги
print("Итоги")
print("Лучше всего отработала модель", AllModels[numpy.argmin(MSEOfModels)], "с среднеквадратической ошибкой", MSEOfModels[numpy.argmin(MSEOfModels)])

Итоги
Лучше всего отработала модель KNeighborsRegressor с среднеквадратической ошибкой 4.753353508139491
